In [1]:
import mitiq
mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [2]:
from cirq import LineQubit, Circuit, rx, rz, CNOT

a, b = LineQubit.range(2)
circuit = Circuit(
    rx(0.1).on(a),
    rx(0.1).on(a),
    rz(0.4).on(a),
    rx(-0.72).on(a),
    rz(0.2).on(a),
    rx(-0.8).on(b),
    CNOT.on(a, b),
)

print(circuit)

0: ───Rx(0.032π)────Rx(0.032π)───Rz(0.127π)───Rx(-0.229π)───Rz(0.064π)───@───
                                                                         │
1: ───Rx(-0.255π)────────────────────────────────────────────────────────X───


In [3]:
import numpy as np
from cirq import DensityMatrixSimulator, amplitude_damp
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.1):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with amplitude damping noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(amplitude_damp(gamma=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [4]:
# Compute the expectation value of the |0><0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.3}")

Error without mitigation: 0.0753


In [5]:
from mitiq import ddd

rule = ddd.rules.yy

In [6]:
circuits_with_ddd = ddd.generate_circuits_with_ddd(circuit=circuit, rule=rule)
print(circuits_with_ddd[0])

0: ───Rx(0.032π)────Rx(0.032π)───Rz(0.127π)───Rx(-0.229π)───Rz(0.064π)───@───
                                                                         │
1: ───Rx(-0.255π)───I────────────Y────────────Y─────────────I────────────X───


In [7]:
mitigated_result = ddd.execute_with_ddd(
    circuit=circuit,
    executor=execute,
    rule=rule,
)

In [8]:
print(f"Error with mitigation (DDD): {abs(ideal_value - mitigated_result) :.3}")

Error with mitigation (DDD): 0.00743


In [48]:
from mitiq import ddd
circuits_with_ddd = ddd.generate_circuits_with_ddd( circuit, rule=rule, num_trials=3)

In [50]:
mitigated_result_1 = ddd.execute_with_ddd(
    circuit=circuits_with_ddd[0],
    executor=execute,
    rule=rule,
)
circuits_with_ddd[0]

0: ───Rx(0.032π)────Rx(0.032π)───Rz(0.127π)───Rx(-0.229π)───Rz(0.064π)───@───
                                                                         │
1: ───Rx(-0.255π)───I────────────Y────────────Y─────────────I────────────X───

In [39]:
mitigated_result_2 = ddd.execute_with_ddd(
    circuit=circuits_with_ddd[1],
    executor=execute,
    rule=rule,
)
circuits_with_ddd[1]

0: ───Rx(0.032π)────Rx(0.032π)───Rz(0.127π)───Rx(-0.229π)───Rz(0.064π)───@───
                                                                         │
1: ───Rx(-0.255π)───I────────────Y────────────Y─────────────I────────────X───

In [40]:
mitigated_result_3 = ddd.execute_with_ddd(
    circuit=circuits_with_ddd[2],
    executor=execute,
    rule=rule,
)
circuits_with_ddd[2]

0: ───Rx(0.032π)────Rx(0.032π)───Rz(0.127π)───Rx(-0.229π)───Rz(0.064π)───@───
                                                                         │
1: ───Rx(-0.255π)───I────────────Y────────────Y─────────────I────────────X───

In [41]:
mitigated_results = [mitigated_result_1, mitigated_result_2, mitigated_result_3]
combined_result = ddd.combine_results(mitigated_results)
combined_result

0.7953312397003174

In [42]:
print(f"Error with mitigation (DDD): {abs(ideal_value - combined_result) :.3}")

Error with mitigation (DDD): 0.00743
